In [1]:
import pandas as pd

#### leverage logistic regression as this is classification

In [2]:
#load in data
df = pd.read_csv('lagging.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (9,10,11,12,13,14,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#import classification, traintestsplit, and counter
from collections import Counter
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
firesizecriteria=[]
for i,y in enumerate(df['FIRE SIZE MAX']):
    if i%100000==0:
        print(i)
    #if (float(y)-500) > float(df1000['diffmax'][i]):
   

    if df['diffmax'][i] > 20000:
            firesizecriteria.append(1)
    elif df['diffmax'][i]*5 < df['FIRE SIZE MAX'][i]:
        if df['diffmax'][i] > 600:
            firesizecriteria.append(1)
        else:
            firesizecriteria.append(0)
    elif df['diffmax'][i]*2 < float(df['FIRE SIZE MAX'][i]):
        if df['diffmax'][i] > 1000:
            firesizecriteria.append(1)
        else:
            firesizecriteria.append(0)

            
    elif df['diffmax'][i]*4 < float(df['FIRE SIZE MAX'][i]):
        if df['diffmax'][i] > 500:
            firesizecriteria.append(1)
        else:
            firesizecriteria.append(0)
    elif df['diffmax'][i]*100 < float(df['FIRE SIZE MAX'][i]):
        if df['diffmax'][i] > 200:
            firesizecriteria.append(1)
        else:
            firesizecriteria.append(0)
    
    else:
        firesizecriteria.append(0)

#     else:
#         print(0)   
#         firesizecriteria.append(0)

0
100000
200000


In [5]:
df5 = pd.get_dummies(df['day of week'], dummy_na=0)

In [6]:
df=pd.concat([df,df5], axis=1)

In [7]:
df['firesize']=pd.DataFrame(firesizecriteria)

##### X = df.drop(columns=['lat lon', 'day of week','day_of_month','level_0','index','A','diffmax','day','dates','B','C','D','E','F','G','longitude','FIRE SIZE MAX','firesize','fire binary','FIRE SIZE SUM', 'cause', 'sz_class', 'state', 'latitude', 'Unnamed: 0'])

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = X
scaler.fit(data)
X2=scaler.transform(data)

NameError: name 'X' is not defined

In [9]:
y = df2=['firesize']

In [10]:
X2, y = make_classification(n_classes=2, class_sep=2, weights=[.7,.3], n_samples=2000 )

In [11]:
from imblearn.under_sampling import RandomUnderSampler # doctest: +NORMALIZE_WHITESPACE
rus = RandomUnderSampler(ratio = .4)
rus.fit(X2, y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


RandomUnderSampler(random_state=None, ratio=0.4, replacement=False,
          return_indices=False)

In [12]:
## train test split
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=.4, random_state = 18)

In [13]:
pd.DataFrame(y_test)[0].value_counts()

0    561
1    239
Name: 0, dtype: int64

In [14]:
from sklearn.linear_model import LogisticRegression, LinearRegression

#### start logistic regression

In [15]:
#import various keras models and features
from keras.models import Sequential
from keras.layers import Dense

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [16]:
#transform yvariables into categorical variables like dummy variables
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [17]:
model = Sequential()

In [18]:
#create variables essential to plugging into the model
n_input = X_train.shape[1]
n_hidden = n_input
n_output = y_train.shape[1]

In [19]:
#add key features
model.add(Dense(n_hidden, input_dim=n_input, activation='relu'))
model.add(Dense(n_output, activation='softmax'))

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=10, batch_size=None)#, batch_size=None, verbose=0)

Train on 1200 samples, validate on 800 samples
Epoch 1/10
1200/1200 [==============================] - 0s 307us/step - loss: 0.8769 - acc: 0.5617 - val_loss: 0.6275 - val_acc: 0.6700
Epoch 2/10
1200/1200 [==============================] - 0s 46us/step - loss: 0.5219 - acc: 0.7300 - val_loss: 0.3880 - val_acc: 0.8075
Epoch 3/10
1200/1200 [==============================] - 0s 48us/step - loss: 0.3353 - acc: 0.8483 - val_loss: 0.2551 - val_acc: 0.8988
Epoch 4/10
1200/1200 [==============================] - 0s 49us/step - loss: 0.2249 - acc: 0.9233 - val_loss: 0.1750 - val_acc: 0.9400
Epoch 5/10
1200/1200 [==============================] - 0s 46us/step - loss: 0.1591 - acc: 0.9592 - val_loss: 0.1240 - val_acc: 0.9725
Epoch 6/10
1200/1200 [==============================] - 0s 53us/step - loss: 0.1190 - acc: 0.9800 - val_loss: 0.0930 - val_acc: 0.9838
Epoch 7/10
1200/1200 [==============================] - 0s 68us/step - loss: 0.0951 - acc: 0.9875 - val_loss: 0.0728 - val_acc: 0.9962
Epoch 8

In [22]:
from sklearn.metrics import classification_report, confusion_matrix

In [23]:
#build a confusion matrix. left is jump, 1 is is no jump
confusion_matrix(np.argmax(y_test, axis=1), model.predict_classes(X_test))

array([[560,   1],
       [  3, 236]], dtype=int64)